# Playing with the Django ORM in Jupyter

First, import the models:

In [1]:
from trees.models import Tree, Species

Let's use a Django ORM annotation to count the number of trees for each species:

In [2]:
from django.db.models import Count

for s in Species.objects.annotate(
    num_trees = Count('tree')
).order_by('-num_trees'):
    print(s.name, s.num_trees)

Morera 18809
Fresno europeo 8037
Pltano 4319
Paraiso 4064
N/D 2831
Fresno americano 2022
Acacia SP 1038
Acer 951
Paraiso sombrilla 809
Olmo comun 776
Caducifolio 650
Prunas 558
Jacarand 523
Perenne 470
Aguaribay 423
Olmo bola 323
Ailanthus 299
Conifera 252
Ligustro 202
Tilo 196
lamo blanco 167
Tipa 127
Braquiquito 109
lamo criollo 108
Acacia visco 108
Liquidambar 99
Palo borracho 64
Catalpa 45
Eucalyptus 26
Algarrobo 7
Arabia 4
rbol del cielo 2
Maiten 1


Here's the SQL that was used for that query:

In [3]:
from django.db import connection
connection.queries[-1]

{'sql': 'SELECT "trees_species"."id", "trees_species"."name", COUNT("trees_tree"."id") AS "num_trees" FROM "trees_species" LEFT OUTER JOIN "trees_tree" ON ("trees_species"."id" = "trees_tree"."species_id") GROUP BY "trees_species"."id", "trees_species"."name" ORDER BY "num_trees" DESC',
 'time': '0.045'}

Let's loop through the first 20 trees and display them:

In [4]:
for tree in Tree.objects.all()[:20]:
    print(tree.latitude, tree.longitude, tree.species.name)

-32.8747311819 -68.8404148439 Jacarand
-32.8746870145999 -68.8403995242999 Jacarand
-32.8746428589999 -68.8403934432 Jacarand
-32.8959944483 -68.8574212648999 Fresno europeo
-32.8959160026 -68.8574162229 Fresno europeo
-32.8958575989 -68.8574080471999 Morera
-32.8958265686 -68.8573570714 Fresno europeo
-32.8958308492999 -68.8572981214 Morera
-32.895838834 -68.8572185353999 Morera
-32.8958462025 -68.8571426345999 Morera
-32.8958536317 -68.8570828522999 Morera
-32.8958572817999 -68.8570150631999 Morera
-32.8958654194 -68.8569486959 Morera
-32.8958690676999 -68.8568794317999 Fresno americano
-32.8958764401999 -68.8568072161999 Morera
-32.8958801479 -68.8567599137 Morera
-32.8958850495999 -68.8567009637 N/D
-32.8958905758999 -68.8566442208 Morera
-32.8958954910999 -68.8565970585 Morera
-32.8958979196999 -68.8565423330999 Fresno americano


This was actually very inefficient: it used 21 SQL queries!

In [7]:
connection.queries[1:]

[{'sql': 'SELECT "trees_tree"."id", "trees_tree"."species_id", "trees_tree"."latitude", "trees_tree"."longitude" FROM "trees_tree" LIMIT 20',
  'time': '0.006'},
 {'sql': 'SELECT "trees_species"."id", "trees_species"."name" FROM "trees_species" WHERE "trees_species"."id" = 1',
  'time': '0.000'},
 {'sql': 'SELECT "trees_species"."id", "trees_species"."name" FROM "trees_species" WHERE "trees_species"."id" = 1',
  'time': '0.000'},
 {'sql': 'SELECT "trees_species"."id", "trees_species"."name" FROM "trees_species" WHERE "trees_species"."id" = 1',
  'time': '0.000'},
 {'sql': 'SELECT "trees_species"."id", "trees_species"."name" FROM "trees_species" WHERE "trees_species"."id" = 2',
  'time': '0.000'},
 {'sql': 'SELECT "trees_species"."id", "trees_species"."name" FROM "trees_species" WHERE "trees_species"."id" = 2',
  'time': '0.000'},
 {'sql': 'SELECT "trees_species"."id", "trees_species"."name" FROM "trees_species" WHERE "trees_species"."id" = 3',
  'time': '0.000'},
 {'sql': 'SELECT "tree

If we use `.select_related('trees')` we can drop that down to just a single query:

In [9]:
for tree in Tree.objects.select_related('species')[:20]:
    print(tree.latitude, tree.longitude, tree.species.name)

-32.8747311819 -68.8404148439 Jacarand
-32.8746870145999 -68.8403995242999 Jacarand
-32.8746428589999 -68.8403934432 Jacarand
-32.8959944483 -68.8574212648999 Fresno europeo
-32.8959160026 -68.8574162229 Fresno europeo
-32.8958575989 -68.8574080471999 Morera
-32.8958265686 -68.8573570714 Fresno europeo
-32.8958308492999 -68.8572981214 Morera
-32.895838834 -68.8572185353999 Morera
-32.8958462025 -68.8571426345999 Morera
-32.8958536317 -68.8570828522999 Morera
-32.8958572817999 -68.8570150631999 Morera
-32.8958654194 -68.8569486959 Morera
-32.8958690676999 -68.8568794317999 Fresno americano
-32.8958764401999 -68.8568072161999 Morera
-32.8958801479 -68.8567599137 Morera
-32.8958850495999 -68.8567009637 N/D
-32.8958905758999 -68.8566442208 Morera
-32.8958954910999 -68.8565970585 Morera
-32.8958979196999 -68.8565423330999 Fresno americano


In [10]:
connection.queries[-1]

{'sql': 'SELECT "trees_tree"."id", "trees_tree"."species_id", "trees_tree"."latitude", "trees_tree"."longitude", "trees_species"."id", "trees_species"."name" FROM "trees_tree" INNER JOIN "trees_species" ON ("trees_tree"."species_id" = "trees_species"."id") LIMIT 20',
 'time': '0.000'}